<a href="https://colab.research.google.com/github/electropavuk/crypto_trader/blob/master/evolution_strategies_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install python-dotenv

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow.keras as keras
import numpy as np

# !pip install python-dotenv

from dotenv import load_dotenv

import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
from copy import deepcopy
sns.set()


pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)

np.set_printoptions(edgeitems=10, linewidth=200)


In [ ]:
google_drive_dir = '/content/drive/MyDrive/Colab Notebooks/crypto_trader/'
load_dotenv(google_drive_dir + '.env')

True

In [ ]:
    def prepare_data(df):
        df = df.drop(columns=['Open time', 'Open', 'Close time', 'Ignore', 'USDT volume'])

        price = df['Close']

        df['Buy / volume ratio'] = df['BTC buy volume'] / df['BTC volume']
        df['Sell / volume ratio'] = 1 - df['Buy / volume ratio']

        MAs = (3, 7, 25, 99)
        for avg in MAs:
            df[f'{avg} MA'] = df['Close'].rolling(avg).mean()

        for attr in ('High', 'Low', 'Close') + tuple(f'{avg} MA' for avg in MAs):
            df[attr].iloc[1:] = (df[attr].iloc[1:] / df[attr].iloc[:-1].values)

        df = df.dropna()


        df = (df - df.min()) / (df.max() - df.min())
        price = price.iloc[-len(df):]

        inputs = df.to_numpy()
        price = price.to_numpy()

        data = [(inputs[t-w:t], price[t-1]) for t in range(w, len(inputs) + 1)]

        return data



    window_size = w = 30
    take = 250
    train = 180
    valid = take - train

    history = pd.read_csv(google_drive_dir + 'data/' + '1d_1y.csv')

    data = prepare_data(history)[-take:]
    train = data[:train]
    valid = data[-valid:]

In [ ]:
class FullModel():
    def __init__(self, layer_size, output_size, features):
        self.layers = []
        for _ in range(features):
            feature_layers = [
                keras.layers.Dense(layer_size, activation='relu'),
                keras.layers.Dropout(.4),
                keras.layers.Dense(output_size, activation='softmax'),
            ]
            self.layers += feature_layers
            self.layers_per_feature = len(feature_layers)
    
    def call(self, inputs):
        decision = np.zeros(3)
        for i, feature in enumerate(inputs.T):
            feature = feature.reshape((1, -1))
            for layer in self.layers[i * self.layers_per_feature: (i + 1) * self.layers_per_feature]:
                feature = layer(feature)
            decision += feature
        return decision


    def get_vector_weights(self):
        def helper(data):
            if isinstance(data, np.ndarray):
                flat.append(data.reshape(-1))
            else:
                for d in data:
                    helper(d)
        flat = []
        helper([layer.get_weights() for layer in self.layers])
        return np.hstack(flat)


    def set_vector_weights(self, weights):
        weights = np.array(weights)
        idx = 0
        for layer in self.layers:
            W = layer.get_weights()
            for i, w in enumerate(W):
                take = np.prod(w.shape)
                W[i] = weights[idx:idx+take].reshape(w.shape)
                idx += take

    


    def act(self, inputs):
        # 0 wait, 1 buy, 2 sell
        return np.argmax(self.call(inputs))

In [ ]:
def fitness(weights):
    model.set_vector_weights(weights)
    data = train if training else valid

    init_money = balance = 1000
    quantity = 0
    for inputs, price in data:
        action = model.act(inputs)
        if action == 1 and balance > 0:
            quantity = balance / price
            balance = 0
        elif action == 2 and quantity > 0:
            balance = quantity * price
            quantity = 0
    
    if quantity > 0:
        balance += quantity * price
    
    profit = (balance - init_money) / init_money
    if not profit:
        profit = -999
    return profit + random.random() / 10

def objective_function(batch):
    scores = [fitness(sample) for sample in batch]
    print('profit {:>5.1f} %'.format(-min(scores)))
    return tf.convert_to_tensor(scores)

In [ ]:
features = 13
training = True

model = FullModel(layer_size=500, output_size=3, features=features)
model.call(train[0][0])



optim_results = tfp.optimizer.differential_evolution_minimize(
        objective_function,
        initial_position=model.get_vector_weights(),
        population_size=50,
        population_stddev=2.0,
        max_iterations=100,
      )

profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0.4 %
profit  -0

In [ ]:

print(optim_results.converged)
print(optim_results.position)  # Should be (close to) [pi, pi].
print(optim_results.objective_value) 

training = False
w = optim_results.position
print(w)
fitness(w)


tf.Tensor(False, shape=(), dtype=bool)
tf.Tensor(
[ 1.7276766   1.9704201  -1.4302979  -6.018544    3.174273   -6.012249   -5.207326   -1.5477785  -0.15707088 -2.5530274  ... -0.14488506  0.8027284  -1.4797857   1.1499414  -3.8086362   2.7464767
 -1.9625455  -0.21839017 -8.741754   -2.8465767 ], shape=(221039,), dtype=float32)
tf.Tensor(0.40841618513447203, shape=(), dtype=float64)
tf.Tensor(
[ 1.7276766   1.9704201  -1.4302979  -6.018544    3.174273   -6.012249   -5.207326   -1.5477785  -0.15707088 -2.5530274  ... -0.14488506  0.8027284  -1.4797857   1.1499414  -3.8086362   2.7464767
 -1.9625455  -0.21839017 -8.741754   -2.8465767 ], shape=(221039,), dtype=float32)


-0.14536678973263903

In [ ]:
population_size = 40
# With an initial population and a multi-part state.
initial_population = (tf.random.normal([population_size]),
                    tf.random.normal([population_size]))
def easom_fn(x, y):
    return -(tf.math.cos(x) * tf.math.cos(y) *
                tf.math.exp(-(x-np.pi)**2 - (y-np.pi)**2))

optim_results = tfp.optimizer.differential_evolution_minimize(
    easom_fn,
    initial_population=initial_population,
    seed=43210)

print(optim_results.converged)
print(optim_results.position)  # Should be (close to) [pi, pi].
print(optim_results.objective_value)    # Should be -1.

print(easom_fn(*initial_population))


# With a single starting point
initial_position = (tf.constant(1.0), tf.constant(1.0))

optim_results = tfp.optimizer.differential_evolution_minimize(
    easom_fn,
    initial_position=initial_position,
    population_size=40,
    population_stddev=2.0,
    seed=43210)

tf.Tensor(True, shape=(), dtype=bool)
[<tf.Tensor: shape=(), dtype=float32, numpy=3.1416445>, <tf.Tensor: shape=(), dtype=float32, numpy=3.14157>]
tf.Tensor(-1.0, shape=(), dtype=float32)
tf.Tensor(
[-7.4490298e-09  2.8647489e-15 -7.1005792e-09 -2.1226508e-15 -8.1318504e-08 -3.4234626e-11  2.6547955e-09  3.7549718e-04 -6.8245197e-07 -9.6095571e-13 -3.3695210e-07 -3.8771283e-08 -1.1250377e-13
 -1.7597280e-16  5.3460351e-03 -3.1635648e-07 -1.6792746e-09 -6.1077678e-08 -6.0435532e-11 -4.2176646e-07 -3.9743696e-17 -3.4175505e-12 -2.3994505e-13  6.2495732e-22  2.4285504e-07  8.5660513e-19
 -6.9637378e-17  1.3095888e-05 -6.3188998e-12  1.4950092e-20 -1.3358692e-11 -2.6070478e-05 -7.0038944e-08 -1.7993736e-07 -7.5549781e-07 -4.3043150e-10 -2.7431645e-06 -8.4841552e-08 -2.5079158e-17
  3.1501994e-08], shape=(40,), dtype=float32)
